# Building RAG and Agents Applications

## Imports API Keys and Libraries

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [5]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [6]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

## Utilities

In [7]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter, PDFFileLoader
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [8]:
import nest_asyncio
nest_asyncio.apply()

## Simple RAG Pipeline with html files

### Retrieval

In [9]:
#from langchain.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import DirectoryLoader

# Load PDF from data folder
# pdf_path = "data/robert-hand-planets-in-transitspdf-pdf-free.pdf"

# Load all HTML files from data folder
loader = DirectoryLoader("data/", glob="*.html", show_progress=True)
docs = loader.load()

  7%|▋         | 1/15 [00:14<03:24, 14.58s/it]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
 20%|██        | 3/15 [00:14<00:46,  3.84s/it]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
 33%|███▎      | 5/15 [00:14<00:19,  1.90s/it]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
 47%|████▋     | 7/15 [00:15<00:09,  1.13s/it]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is

In [10]:
docs

[Document(metadata={'source': 'data/Synastry Art and Science- Synastry Aspects, Element Compatibility, House Overlays.html'}, page_content='Astrolibrary logo\n\nAstroLibrary 🔎\n\nDaily Personalized Horoscope ↗\n\nSynastry Art and Science- Synastry Aspects, Element Compatibility, House Overlays\n\n(This is a synastry lesson. If you prefer, see Synastry Interpretations instead.)\n\nWhat is synastry? Synastry is the art and science of comparing two birth charts to ascertain the energy flow between two individuals. Synastry is the astrology of relationships, and it reveals the level of compatibility between two people. Please don\'t confuse this with fortune-telling. Synastry will not tell you if you are "fated to be together." Any two people can choose to be together. Synastry shows if two unique individuals can be themselves and blend nicely together, or if being themselves causes the irritating grinding of mismatched gears.\n\nSpecifically, 3 things are compared:\n\nElements – At the ve

In [11]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o-mini").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = tiktoken_len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Split on paragraphs, then sentences
)

split_chunks = text_splitter.split_documents(docs)

In [12]:
split_chunks

[Document(metadata={'source': 'data/Synastry Art and Science- Synastry Aspects, Element Compatibility, House Overlays.html'}, page_content='Astrolibrary logo\n\nAstroLibrary 🔎\n\nDaily Personalized Horoscope ↗\n\nSynastry Art and Science- Synastry Aspects, Element Compatibility, House Overlays\n\n(This is a synastry lesson. If you prefer, see Synastry Interpretations instead.)\n\nWhat is synastry? Synastry is the art and science of comparing two birth charts to ascertain the energy flow between two individuals. Synastry is the astrology of relationships, and it reveals the level of compatibility between two people. Please don\'t confuse this with fortune-telling. Synastry will not tell you if you are "fated to be together." Any two people can choose to be together. Synastry shows if two unique individuals can be themselves and blend nicely together, or if being themselves causes the irritating grinding of mismatched gears.\n\nSpecifically, 3 things are compared:\n\nElements – At the ve

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="astrology101",
    #collection_name="planets-in-transit",
)

In [14]:
qdrant_retriever = qdrant_vectorstore.as_retriever(search_kwargs={"k": 5})

Setup Graph for retreival node

In [15]:
def retrieve(state):
  retrieved_docs = qdrant_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

In [16]:
from langchain_core.prompts import ChatPromptTemplate


RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are an expert astrologer with deep knowledge of planetary transits and interpretations. Use the provided context from the astrological text to answer the question. 


Guidelines:
1. Focus on the astrological significance and interpretations
2. Explain planetary positions, aspects, and their effects
3. If discussing transits, include timing and duration of influences
4. If the context doesn't contain enough information, say "I don't have enough information as my cosmic sources aren’t aligning on that question right now.”
5. If the question is about a specific chart or transit, make sure to reference the relevant planetary positions and aspects

Provide a detailed astrological interpretation based on the context. Ensure that format is easy to read and understand.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation


In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-mini")

Set up graph for generation node

In [18]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

In [19]:
from langgraph.graph import START, StateGraph, END
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Putting nodes and edges together
(Start -> Retrieve -> Generate -> End)

In [20]:
graph_builder = StateGraph(State)
# Define nodes
graph_builder.add_node("retrieve", retrieve)
graph_builder.add_node("generate", generate)
# Define edges
graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve", "generate")
graph_builder.add_edge("generate", END)
# Compile
graph = graph_builder.compile()

In [21]:
response =graph.invoke({"question" : "What does Mars in 5th mean?"})

In [22]:
response

{'question': 'What does Mars in 5th mean?',
 'context': [Document(metadata={'source': 'data/Planets in Astrology – Lesson 5 _ AstroLibrary.html', '_id': '0af75eef9c8c4a9b8684f989fbf470e1', '_collection_name': 'astrology101'}, page_content="Interpretations for Venus in each zodiac sign are available here: Venus in the Signs\n\nInterpretations for Venus in each House of the birth chart: Venus in Houses\n\nReturn to Top Return to Taurus Return to Libra\n\nastrology planet mars real image\n\nMars\n\n“I act”\n\nBest quality: Initiative Worst quality: Harshness\n\nMars is the exact opposite of Venus. Whereas Venus represents the female, Mars is all male. Whereas Venus is about harmony, Mars is conflict, aggression, and outright war. Mars is the planet of physical energy. It governs your sex drive, your forcefulness and your aggression. It is associated with your desires and aspirations for emotional, physical, mental and spiritual things that stimulate activity. You have to want something be

## Agent Tools

### 1. TavilySearchResults tool

In [23]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)

### 2. Birthchart tool 

In [24]:
from datetime import datetime
from typing import Optional, Dict, Any, Literal, Tuple
from dataclasses import dataclass
from astrology.chart import LocationGeocoder, AstrologicalChart
import pytz
from geopy.exc import GeocoderTimedOut
from dateutil import parser
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from langchain_core.tools import tool

@dataclass
class BirthChartResult:
    """Structured result of birth chart generation."""
    success: bool
    chart_text: Optional[str] = None
    error_type: Optional[Literal["invalid_input", "location_error", "calculation_error", "system_error"]] = None
    error_message: Optional[str] = None
    raw_data: Optional[Dict[str, Any]] = None

def parse_date_time(date_str: str, time_str: str) -> Tuple[Optional[datetime], Optional[str]]:
    """
    Parse various date and time formats into a datetime object.
    """
    try:
        # Try to parse the date using dateutil
        date = parser.parse(date_str)
        
        # Try to parse the time
        time = parser.parse(time_str)
        
        # Combine date and time
        combined_datetime = datetime.combine(date.date(), time.time())
        
        return combined_datetime, None
        
    except Exception as e:
        return None, f"Could not parse date/time: {str(e)}"

def get_location_timezone(location_name: str) -> Tuple[Optional[float], Optional[str]]:
    """
    Get timezone offset for any location using geocoding and timezonefinder.
    """
    try:
        # Create geocoder instance
        geolocator = Nominatim(user_agent="my_astrology_app")
        
        # Get location data
        location = geolocator.geocode(location_name)
        if not location:
            return None, f"Could not find location: {location_name}"
        
        # Get coordinates
        lat, lon = location.latitude, location.longitude
        
        # Use timezonefinder to get timezone
        tf = TimezoneFinder()
        timezone_str = tf.timezone_at(lat=lat, lng=lon)
        
        if not timezone_str:
            return None, f"Could not determine timezone for location: {location_name}"
        
        # Get timezone object
        tz = pytz.timezone(timezone_str)
        
        # Get current offset
        offset = tz.utcoffset(datetime.now()).total_seconds() / 3600
        
        return offset, None
        
    except GeocoderTimedOut:
        return None, "Geocoding service timed out. Please try again."
    except Exception as e:
        return None, f"Error determining timezone: {str(e)}"

def generate_birthchart(
    location_name: str,
    birth_date: str,
    birth_time: str,
    timezone: Optional[float] = None
) -> BirthChartResult:
    """
    Generate an astrological birth chart based on location, date, time, and timezone.
    """
    try:
        # Input validation
        if not location_name or not isinstance(location_name, str):
            return BirthChartResult(
                success=False,
                error_type="invalid_input",
                error_message="Location name must be a non-empty string"
            )
        
        # Parse date and time
        birth_datetime, parse_error = parse_date_time(birth_date, birth_time)
        if parse_error:
            return BirthChartResult(
                success=False,
                error_type="invalid_input",
                error_message=parse_error
            )
        
        # Get timezone if not provided
        if timezone is None:
            timezone, tz_error = get_location_timezone(location_name)
            if tz_error:
                return BirthChartResult(
                    success=False,
                    error_type="location_error",
                    error_message=tz_error
                )
        
        # Create instances of the required classes
        geocoder = LocationGeocoder(user_agent="my_astrology_app")
        chart = AstrologicalChart(geocoder)
        
        # Calculate the chart
        chart_data = chart.calculate(location_name, birth_datetime, timezone)
        
        if chart_data:
            formatted_chart = chart.format_chart(chart_data)
            return BirthChartResult(
                success=True,
                chart_text=formatted_chart,
                raw_data=chart_data
            )
        else:
            return BirthChartResult(
                success=False,
                error_type="location_error",
                error_message="Could not calculate chart. Please check the location name and try again."
            )
            
    except Exception as e:
        return BirthChartResult(
            success=False,
            error_type="system_error",
            error_message=f"An unexpected error occurred: {str(e)}"
        )

@tool
def birthchart_tool(
    location_name: str,
    birth_date: str,
    birth_time: str,
    timezone: Optional[float] = None
) -> str:
    """
    Tool to generate an astrological birth chart.
    
    This tool can handle various date and time formats and automatically determines
    timezone from location if not provided.
    
    Examples:
    - "Generate birth chart for someone born in Bangkok on May 25, 1994 at 7:20 PM"
    - "What's the birth chart for Tokyo, Japan on 1994-05-25 at 19:20?"
    - "Calculate birth chart for New York, USA, born on 05/25/1994 at 7:20 PM"
    """
    result = generate_birthchart(location_name, birth_date, birth_time, timezone)
    
    if result.success:
        return result.chart_text
    else:
        return f"Error: {result.error_message}"

### 3. Transit Tool

In [25]:
from datetime import datetime, timedelta
from typing import Optional, Dict, Any, Literal, Tuple
from dataclasses import dataclass
from langchain_core.tools import tool
from astrology.transits import (
    calculate_transits,
    find_exact_aspect_time,
    format_transit_report
)

@tool
def transit_tool(
    location_name: str,
    birth_date: str,
    birth_time: str,
    transit_date: Optional[str] = None,
    timezone: Optional[float] = None,
    look_ahead_days: int = 30,
    aspect_to_find: Optional[str] = None,
    question_context: Optional[str] = None
) -> str:
    """
    Tool to calculate transits for a birth chart.
    
    This tool can calculate transits for any date and provide context for specific life questions.
    The AI Agent can use this to analyze astrological influences for future events.
    
    Examples:
    - "What are the transits for my birthday next year?"
    - "Should I get a dog in a month's time?"
    - "Is next week good for starting a new project?"
    - "What are the planetary influences for my career change in 3 months?"
    
    Parameters:
    - location_name: Birth location (e.g., "Bangkok, Thailand")
    - birth_date: Birth date (e.g., "25 May 1994")
    - birth_time: Birth time (e.g., "19:20")
    - transit_date: Any date to calculate transits for (e.g., "next month", "2024-06-11")
    - timezone: Optional timezone offset (e.g., 7.0 for Bangkok)
    - look_ahead_days: Number of days to look ahead for exact aspects
    - aspect_to_find: Optional specific aspect to find (e.g., "Jupiter Sun opposition")
    - question_context: Context of the question being asked
    """
    try:
        # First get the birth chart
        birth_result = generate_birthchart(location_name, birth_date, birth_time, timezone)
        if not birth_result.success:
            return f"Error getting birth chart: {birth_result.error_message}"
        
        # Parse transit date if provided
        transit_datetime = None
        if transit_date:
            try:
                from dateutil import parser
                transit_datetime = parser.parse(transit_date)
            except:
                return f"Could not parse transit date: {transit_date}"
        
        # Calculate transits
        transit_data = calculate_transits(
            birth_result.raw_data['planets'],
            transit_datetime or datetime.now(),
            timezone or birth_result.raw_data['time']['timezone']
        )
        
        # Format the report
        report = format_transit_report(transit_data)
        
        # Add context if provided
        if question_context:
            report = f"Transit Analysis for: {question_context}\n\n" + report
        
        # Add exact aspect time if requested
        if aspect_to_find:
            try:
                planet1, planet2, aspect_type = aspect_to_find.split()
                exact_time = find_exact_aspect_time(
                    birth_result.raw_data['planets'],
                    transit_datetime or datetime.now(),
                    (transit_datetime or datetime.now()) + timedelta(days=look_ahead_days),
                    planet1,
                    planet2,
                    aspect_type,
                    timezone or birth_result.raw_data['time']['timezone']
                )
                if exact_time:
                    report += f"\n\nExact {aspect_type} between {planet1} and {planet2} will occur at: {exact_time}"
            except:
                pass
        
        return report
            
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

### 4. Knowledge base retrieval tool

In [26]:
from typing import Annotated, List, Tuple, Union
from langchain_core.tools import tool

@tool
def retrieve_information(
    query: Annotated[str, "query to ask the retrieve information tool"]
    ):
  """Use Retrieval Augmented Generation to retrieve information about the 'Planets in Transit' interpretation."""
  return graph.invoke({"question" : query})

## Constructing Agents

In [27]:
from typing import Any, Callable, List, Optional, TypedDict, Union

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph

### Agent Definition

In [28]:
#wrap each agent into a node

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


def create_agent(
    llm: ChatOpenAI,
    tools: list,
    system_prompt: str,
    team_members: list[str] 
) -> str:
    """Create a function-calling agent and add it to the graph."""

    system_prompt += ("\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification."
    " Your other team members (and other teams) will collaborate with you with their own specialties."
    " You are chosen for a reason! You are one of the following team members: {team_members}.")
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    ).partial(team_members=", ".join(team_members))  # Apply partial here
    # Define team members once
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

### Supervisor Definition

In [29]:
def create_team_supervisor(llm: ChatOpenAI, system_prompt, members: list[str]) -> str:
    """An LLM-based router."""
    options = ["FINISH"] + members
    
    # Define the function schema
    function_def = {
        "name": "route",
        "description": "Select the next role to handle the request.",
        "parameters": {
            "type": "object",
            "properties": {
                "next": {
                    "type": "string",
                    "enum": options,
                    "description": "The next team member to process the request"
                }
            },
            "required": ["next"]
        }
    }

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of: {options}. "
                "You MUST use the route function to make your selection."
            ),
        ]
    ).partial(options=str(options), team_members=", ".join(members))

    # Configure LLM to always use the route function
    llm_with_function = llm.bind(
        function_call={"name": "route"},  # Force it to use route
        functions=[function_def]  # Provide the function definition
    )

    return (
        prompt
        | llm_with_function
        | JsonOutputFunctionsParser()
    )

### Initialise FortuneTeller Team State

In [30]:
import functools
import operator

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI
import functools

class FortuneTellingTeam (TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: str

In [31]:
llm = ChatOpenAI(model="gpt-4.1-mini")

### Create 4 Agents 

In [32]:
TEAM_MEMBERS = ["WorldOracle", "BirthChartCalculator", "TransitAnalyst", "AstrologicalInterpreter"]

# Agent 1: Oracle
oracle_agent = create_agent(
    llm,
    [tavily_tool],
    f"""You are the search specialist in a team of fortune tellers.
    Guidelines:
    - ALWAYS check the current date before searching
    - When a user asks about "this year", determine if they mean:
        * The current year ({datetime.now().year})
        * Or the next 12 months from now
    - For questions about "next year", look for information about {datetime.now().year + 1}
    
    Search Strategy:
    - Include the specific year in your searches
    - For year-spanning predictions (like "next 12 months"), search for both years
    - Format dates clearly in responses:
        "From {datetime.now().strftime('%B %Y')} onwards..."
        "For the remainder of {datetime.now().year}..."
        "Looking ahead to {datetime.now().year + 1}..."
    
    - If information for current dates isn't available:
        * State when the most recent reliable information is from
        * Focus on longer-term transits that are currently active
        * Be transparent about the time range of your information
    
    Remember: Astrology is time-sensitive - always verify and specify the time period you're discussing.""",
    team_members=TEAM_MEMBERS
    )
worldoracle_node = functools.partial(agent_node, agent=oracle_agent, name="WorldOracle")

# Agent 2: Birthchart Calculator
birthchart_calculator = create_agent(
    llm,
    [birthchart_tool],
    """You are a birth chart calculator. Your job is to generate accurate astrological birth charts based on the user's birth location, date, and time. \
    Use the birthchart_tool to perform all calculations. Do not provide interpretations or advice—just return the calculated birth chart data and planetary positions. \
    
    Guidelines:
    - If birth details are NOT provided in the question:
      * Ask ONCE for birth details
      * If not provided, say "Since birth details aren't available yet, I'll let our team provide general insights first."
      * Let other team members handle general insights
    
    - If birth details ARE provided:
      * Generate the birth chart using the birthchart_tool
      * Format output exactly as shown in the example
    
    Remember: Don't keep asking for birth details repeatedly. Users may want general insights first.

    Example Output: 
        Location: Bangkok 
        Latitude: 13.7524938°N 
        Longitude: 100.4935089°E 
        Time: 1994-05-25 19:20 (UTC+7.0) 
        Julian Day: 2449498.013888889 \

        Planetary Positions: \
        Sun      64°3'54" (Gemini 4°3'54")
        Moon     249°7'45" (Sagittarius 9°7'45")
        Mercury  86°22'23" (Gemini 26°22'23")
        Venus    95°20'13" (Cancer 5°20'13")
        Mars     31°10'46" (Taurus 1°10'46")
        Jupiter  216°47'53" (Scorpio 6°47'53")
        Saturn   341°43'56" (Pisces 11°43'56")
        Uranus   296°5'54" (Capricorn 26°5'54")
        Neptune  293°6'49" (Capricorn 23°6'49")
        Pluto    236°28'53" (Scorpio 26°28'53")

        Ascendant (Rising Sign):
        Position: 253°54'16" (Sagittarius 13°54'16")

        House Cusps:
        ------------
        house_1  253°54'16" (Sagittarius 13°54'16")
        house_2  283°16'22" (Capricorn 13°16'22")
        house_3  314°30'5" (Aquarius 14°30'5")
        house_4  347°20'8" (Pisces 17°20'8")
        house_5  19°12'56" (Aries 19°12'56")
        house_6  47°55'17" (Taurus 17°55'17")
        house_7  73°54'16" (Gemini 13°54'16")
        house_8  103°16'22" (Cancer 13°16'22")
        house_9  134°30'5" (Leo 14°30'5")
        house_10 167°20'8" (Virgo 17°20'8")
        house_11 199°12'56" (Libra 19°12'56")
        house_12 227°55'17" (Scorpio 17°55'17")

        Angles:
        -------
        ascendant  253°54'16" (Sagittarius 13°54'16")
        midheaven  167°20'8" (Virgo 17°20'8")
        armc       168°21'3" (Virgo 18°21'3")
        vertex     145°46'40" (Leo 25°46'40")""",
        team_members=TEAM_MEMBERS
)
birthchart_node = functools.partial(agent_node, agent=birthchart_calculator, name="BirthChartCalculator")

# Agent 3: Transit Analyst
transit_analyst = create_agent(
    llm,
    [transit_tool],
    """You are a transit analyst. Your job is to analyze astrological transits and their effects on people's lives.
    Use the transit tool to perform all calculations. Do not provide interpretations or advice—just return the calculated transit data.
    Format your output exactly as in the example below:
        Example 1: Basic transit calculation
        Transit Report for 2025-05-11 14:41
        ==================================================

        Transiting Planets:
        --------------------
        Sun      50°52'23" (Taurus 20°52'23") 
        Moon     215°46'4" (Scorpio 5°46'4") 
        Mercury  31°22'19" (Taurus 1°22'19") 
        Venus    6°58'27" (Aries 6°58'27") 
        Mars     130°28'56" (Leo 10°28'56") 
        Jupiter  83°29'33" (Gemini 23°29'33") 
        Saturn   358°50'37" (Pisces 28°50'37") 
        Uranus   56°54'35" (Taurus 26°54'35") 
        Neptune  1°23'43" (Aries 1°23'43") 
        Pluto    303°48'29" (Aquarius 3°48'29") R

        Aspects to Natal Planets:
        --------------------
        Sun quincunx Moon (orb: 1.70°)  
        Sun semisextile Mercury (orb: 2.69°)  
        Sun sextile Venus (orb: 2.91°)  
        Sun sextile Saturn (orb: 5.22°)  
        Sun conjunction Uranus (orb: 7.16°)  
        Sun sextile Neptune (orb: 2.67°)  
        Sun trine Pluto (orb: 0.26°)  R
        Moon trine Venus (orb: 2.16°)  
        Moon trine Mars (orb: 1.35°)  
        Moon trine Neptune (orb: 7.73°)  
        Moon sextile Pluto (orb: 5.32°)  R
        Mercury sextile Mercury (orb: 5.00°)  
        Mercury semisquare Mars (orb: 0.89°)  
        Mercury conjunction Jupiter (orb: 2.88°)  
        Mercury square Saturn (orb: 2.47°)  
        Mercury semisextile Uranus (orb: 0.54°)  
        Mercury square Neptune (orb: 5.02°)  
        Venus semisquare Sun (orb: 0.54°)  
        Venus trine Moon (orb: 0.43°)  
        Venus sextile Mercury (orb: 3.96°)  
        Venus square Venus (orb: 1.64°)  
        Venus square Saturn (orb: 6.49°)  
        Venus square Neptune (orb: 3.94°)  
        Venus quincunx Pluto (orb: 1.53°)  R
        Mars opposition Moon (orb: 4.59°)  
        Mars conjunction Mercury (orb: 0.19°)  
        Mars semisextile Saturn (orb: 2.34°)  
        Mars semisextile Neptune (orb: 0.22°)  
        Mars square Pluto (orb: 2.63°)  R
        Jupiter conjunction Moon (orb: 1.03°) R 
        Jupiter opposition Mercury (orb: 5.43°) R 
        Jupiter quincunx Venus (orb: 0.18°) R 
        Jupiter square Mars (orb: 3.68°) R 
        Jupiter sesquisquare Jupiter (orb: 1.69°) R 
        Jupiter square Pluto (orb: 2.99°) R R
        Saturn trine Moon (orb: 5.96°)  
        Saturn quincunx Mars (orb: 1.25°)  
        Uranus trine Sun (orb: 5.23°) R 
        Uranus square Mercury (orb: 5.27°) R 
        Uranus quincunx Jupiter (orb: 2.61°) R 
        Uranus sextile Saturn (orb: 2.75°) R 
        Uranus trine Uranus (orb: 0.81°) R 
        Uranus sextile Neptune (orb: 5.30°) R 
        Uranus conjunction Pluto (orb: 7.71°) R R
        Neptune trine Sun (orb: 2.24°) R 
        Neptune quincunx Jupiter (orb: 0.38°) R 
        Neptune sextile Saturn (orb: 5.73°) R 
        Neptune trine Uranus (orb: 3.80°) R 
        Pluto opposition Sun (orb: 5.61°) R 
        Pluto quincunx Jupiter (orb: 2.99°) R 
        Pluto trine Saturn (orb: 2.36°) R 
        Pluto opposition Uranus (orb: 0.43°) R 
        Pluto trine Neptune (orb: 4.91°) R 

        Example 2: Future date with context
        Transit Analysis for: getting a pet

        Transit Report for 2024-06-11 00:00
        ==================================================

        Transiting Planets:
        --------------------
        Sun      80°17'49" (Gemini 20°17'49") 
        Moon     131°17'53" (Leo 11°17'53") 
        Mercury  75°22'41" (Gemini 15°22'41") 
        Venus    81°56'42" (Gemini 21°56'42") 
        Mars     31°7'36" (Taurus 1°7'36") 
        Jupiter  63°40'11" (Gemini 3°40'11") 
        Saturn   349°7'35" (Pisces 19°7'35") 
        Uranus   54°42'28" (Taurus 24°42'28") 
        Neptune  359°48'14" (Pisces 29°48'14") 
        Pluto    301°46'9" (Aquarius 1°46'9") R

        Aspects to Natal Planets:
        --------------------
        Sun semisextile Mars (orb: 2.94°)  
        Sun conjunction Jupiter (orb: 0.40°)  
        Sun sextile Neptune (orb: 4.26°)  
        Sun trine Pluto (orb: 2.30°)  R
        Moon trine Moon (orb: 2.17°)  
        Moon opposition Mercury (orb: 6.25°)  
        Moon opposition Jupiter (orb: 5.46°)  
        Mercury conjunction Sun (orb: 6.08°)  
        Mercury semisquare Moon (orb: 0.07°)  
        Mercury conjunction Venus (orb: 4.43°)  
        Mercury sextile Mars (orb: 4.75°)  
        Mercury square Saturn (orb: 7.25°)  
        Mercury semisextile Uranus (orb: 1.67°)  
        Mercury square Neptune (orb: 3.43°)  
        Venus sextile Mars (orb: 4.21°)  
        Venus semisextile Jupiter (orb: 1.67°)  
        Venus square Neptune (orb: 5.53°)  
        Mars semisquare Mercury (orb: 0.80°)  
        Mars conjunction Mars (orb: 0.05°)  
        Mars semisextile Jupiter (orb: 2.49°)  
        Mars semisextile Neptune (orb: 1.38°)  
        Mars square Pluto (orb: 0.59°)  R
        Jupiter sesquisquare Sun (orb: 1.50°) R 
        Jupiter square Moon (orb: 4.50°) R 
        Jupiter sesquisquare Venus (orb: 0.15°) R 
        Jupiter opposition Mars (orb: 5.67°) R 
        Jupiter square Pluto (orb: 5.03°) R R
        Saturn quincunx Moon (orb: 0.43°)  
        Saturn square Mercury (orb: 3.65°)  
        Saturn conjunction Saturn (orb: 7.39°)  
        Uranus square Mars (orb: 5.03°) R 
        Uranus trine Jupiter (orb: 7.57°) R 
        Uranus trine Uranus (orb: 1.39°) R 
        Uranus sextile Neptune (orb: 3.71°) R 
        Uranus conjunction Pluto (orb: 5.67°) R R
        Neptune quincunx Sun (orb: 2.82°) R 
        Neptune quincunx Venus (orb: 1.17°) R 
        Neptune sextile Saturn (orb: 3.99°) R 
        Neptune trine Uranus (orb: 1.59°) R 
        Pluto opposition Jupiter (orb: 7.19°) R 
        Pluto trine Saturn (orb: 7.36°) R 
        Pluto opposition Uranus (orb: 1.77°) R 
        Pluto trine Neptune (orb: 3.32°) R 
        Pluto sextile Pluto (orb: 5.29°) R R

        Example 3: Finding exact aspect
        Transit Report for 2024-05-11 00:00
        ==================================================

        Transiting Planets:
        --------------------
        Sun      50°31'6" (Taurus 20°31'6") 
        Moon     84°13'57" (Gemini 24°13'57") 
        Mercury  24°20'53" (Aries 24°20'53") 
        Venus    43°49'45" (Taurus 13°49'45") 
        Mars     7°43'51" (Aries 7°43'51") 
        Jupiter  56°24'6" (Taurus 26°24'6") 
        Saturn   347°25'55" (Pisces 17°25'55") 
        Uranus   52°56'53" (Taurus 22°56'53") 
        Neptune  359°12'12" (Pisces 29°12'12") 
        Pluto    302°5'20" (Aquarius 2°5'20") R

        Aspects to Natal Planets:
        --------------------
        Sun sextile Mars (orb: 3.67°)  
        Sun conjunction Jupiter (orb: 7.66°)  
        Sun sextile Neptune (orb: 4.86°)  
        Sun trine Pluto (orb: 1.98°)  R
        Moon sesquisquare Mercury (orb: 0.22°)  
        Moon trine Mars (orb: 1.40°)  
        Mercury conjunction Moon (orb: 2.14°)  
        Mercury sextile Mercury (orb: 2.02°)  
        Mercury semisextile Jupiter (orb: 0.03°)  
        Mercury square Neptune (orb: 2.83°)  
        Venus semisquare Sun (orb: 0.18°)  
        Venus square Mars (orb: 2.39°)  
        Venus square Neptune (orb: 6.13°)  
        Mars conjunction Mercury (orb: 6.83°)  
        Mars semisquare Saturn (orb: 1.25°)  
        Mars semisextile Neptune (orb: 1.98°)  
        Mars square Pluto (orb: 0.91°)  R
        Jupiter opposition Venus (orb: 7.03°) R 
        Jupiter quincunx Mars (orb: 0.93°) R 
        Jupiter square Pluto (orb: 4.71°) R R
        Saturn sextile Venus (orb: 2.10°)  
        Saturn conjunction Saturn (orb: 5.70°)  
        Uranus trine Sun (orb: 5.58°) R 
        Uranus quincunx Moon (orb: 1.87°) R 
        Uranus square Mercury (orb: 1.75°) R 
        Uranus trine Jupiter (orb: 0.30°) R 
        Uranus trine Uranus (orb: 3.15°) R 
        Uranus sextile Neptune (orb: 3.10°) R 
        Uranus conjunction Pluto (orb: 5.99°) R R
        Neptune trine Sun (orb: 2.60°) R 
        Neptune quincunx Moon (orb: 1.12°) R 
        Neptune square Mercury (orb: 1.23°) R 
        Neptune trine Jupiter (orb: 3.29°) R 
        Neptune sextile Saturn (orb: 5.68°) R 
        Neptune trine Uranus (orb: 0.17°) R 
        Pluto opposition Sun (orb: 5.96°) R 
        Pluto quincunx Moon (orb: 2.25°) R 
        Pluto quincunx Mercury (orb: 2.13°) R 
        Pluto opposition Jupiter (orb: 0.08°) R 
        Pluto opposition Uranus (orb: 3.53°) R 
        Pluto trine Neptune (orb: 2.72°) R 
        Pluto sextile Pluto (orb: 5.61°) R R

        Example 4: Career question
        Transit Analysis for: career change

        Transit Report for 2024-05-18 00:00
        ==================================================

        Transiting Planets:
        --------------------
        Sun      57°16'22" (Taurus 27°16'22") 
        Moon     171°27'4" (Virgo 21°27'4") 
        Mercury  32°33'48" (Taurus 2°33'48") 
        Venus    52°26'33" (Taurus 22°26'33") 
        Mars     13°4'30" (Aries 13°4'30") 
        Jupiter  58°3'12" (Taurus 28°3'12") 
        Saturn   347°55'58" (Pisces 17°55'58") 
        Uranus   53°21'16" (Taurus 23°21'16") 
        Neptune  359°22'40" (Pisces 29°22'40") 
        Pluto    302°3'6" (Aquarius 2°3'6") R

        Aspects to Natal Planets:
        --------------------
        Sun conjunction Sun (orb: 6.79°)  
        Sun semisextile Mercury (orb: 1.50°)  
        Sun conjunction Jupiter (orb: 6.01°)  
        Sun sextile Neptune (orb: 4.69°)  
        Sun trine Pluto (orb: 2.01°)  R
        Moon trine Mars (orb: 3.95°)  
        Mercury semisextile Sun (orb: 0.90°)  
        Mercury square Moon (orb: 4.92°)  
        Mercury semisextile Jupiter (orb: 1.68°)  
        Mercury square Neptune (orb: 3.00°)  
        Venus sextile Mercury (orb: 2.77°)  
        Venus square Mars (orb: 7.74°)  
        Venus square Neptune (orb: 5.96°)  
        Mars conjunction Mercury (orb: 1.38°)  
        Mars semisquare Saturn (orb: 1.75°)  
        Mars semisextile Neptune (orb: 1.80°)  
        Mars square Pluto (orb: 0.87°)  R
        Jupiter semisquare Moon (orb: 0.35°) R 
        Jupiter opposition Mercury (orb: 4.23°) R 
        Jupiter square Pluto (orb: 4.75°) R R
        Saturn semisextile Mars (orb: 1.34°)  
        Saturn conjunction Saturn (orb: 6.20°)  
        Uranus trine Sun (orb: 1.17°) R 
        Uranus trine Moon (orb: 4.65°) R 
        Uranus square Mercury (orb: 6.47°) R 
        Uranus trine Venus (orb: 3.66°) R 
        Uranus trine Jupiter (orb: 1.95°) R 
        Uranus trine Uranus (orb: 2.74°) R 
        Uranus sextile Neptune (orb: 3.28°) R 
        Uranus conjunction Pluto (orb: 5.95°) R R
        Neptune trine Sun (orb: 4.16°) R 
        Neptune trine Moon (orb: 1.66°) R 
        Neptune trine Venus (orb: 0.67°) R 
        Neptune trine Jupiter (orb: 4.94°) R 
        Neptune sextile Saturn (orb: 5.18°) R 
        Neptune trine Uranus (orb: 0.24°) R 
        Pluto opposition Sun (orb: 0.79°) R 
        Pluto sextile Moon (orb: 5.03°) R 
        Pluto opposition Venus (orb: 4.04°) R 
        Pluto sesquisquare Mars (orb: 1.59°) R 
        Pluto opposition Jupiter (orb: 1.57°) R 
        Pluto opposition Uranus (orb: 3.13°) R 
        Pluto trine Neptune (orb: 2.90°) R 
        Pluto sextile Pluto (orb: 5.57°) R R

        Example 5: Different date formats
        Testing date format: next month
        Could not parse transit date: next month

        Testing date format: 2024-05-25
        Transit Report for 2024-05-25 00:00
        ==================================================

        Transiting Planets:
        --------------------
        Sun      64°0'18" (Gemini 4°0'18") 
        Moon     257°33'32" (Sagittarius 17°33'32") 
        Mercury  42°49'33" (Taurus 12°49'33") 
        Venus    61°2'59" (Gemini 1°2'59") 
        Mars     18°23'9" (Aries 18°23'9") 
        Jupiter  59°42'12" (Taurus 29°42'12") 
        Saturn   348°22'4" (Pisces 18°22'4") 
        Uranus   53°45'31" (Taurus 23°45'31") 
        Neptune  359°31'50" (Pisces 29°31'50") 
        Pluto    301°59'35" (Aquarius 1°59'35") R

        Aspects to Natal Planets:
        --------------------
        Sun conjunction Sun (orb: 0.06°)  
        Sun conjunction Venus (orb: 3.02°)  
        Sun semisquare Mars (orb: 0.68°)  
        Sun conjunction Jupiter (orb: 4.36°)  
        Sun sextile Neptune (orb: 4.53°)  
        Sun trine Pluto (orb: 2.07°)  R
        Moon opposition Sun (orb: 5.12°)  
        Mercury semisquare Mercury (orb: 1.45°)  
        Mercury semisextile Uranus (orb: 2.61°)  
        Mercury square Neptune (orb: 3.16°)  
        Venus semisextile Sun (orb: 1.33°)  
        Venus square Neptune (orb: 5.81°)  
        Mars semisextile Sun (orb: 2.83°)  
        Mars sesquisquare Moon (orb: 1.38°)  
        Mars semisextile Venus (orb: 0.13°)  
        Mars semisextile Jupiter (orb: 1.48°)  
        Mars semisextile Neptune (orb: 1.65°)  
        Mars square Pluto (orb: 0.81°)  R
        Jupiter quincunx Sun (orb: 2.79°) R 
        Jupiter opposition Mercury (orb: 6.03°) R 
        Jupiter square Pluto (orb: 4.81°) R R
        Saturn square Sun (orb: 7.73°)  
        Saturn square Moon (orb: 5.83°)  
        Saturn sextile Mercury (orb: 1.09°)  
        Saturn conjunction Saturn (orb: 6.64°)  
        Uranus trine Sun (orb: 7.91°) R 
        Uranus trine Venus (orb: 4.95°) R 
        Uranus square Mars (orb: 7.71°) R 
        Uranus trine Jupiter (orb: 3.60°) R 
        Uranus trine Uranus (orb: 2.34°) R 
        Uranus sextile Neptune (orb: 3.43°) R 
        Uranus conjunction Pluto (orb: 5.89°) R R
        Neptune trine Venus (orb: 7.94°) R 
        Neptune square Mars (orb: 4.73°) R 
        Neptune trine Jupiter (orb: 6.59°) R 
        Neptune sextile Saturn (orb: 4.75°) R 
        Neptune trine Uranus (orb: 0.64°) R 
        Pluto opposition Sun (orb: 7.52°) R 
        Pluto opposition Venus (orb: 4.57°) R 
        Pluto opposition Jupiter (orb: 3.22°) R 
        Pluto opposition Uranus (orb: 2.72°) R 
        Pluto trine Neptune (orb: 3.05°) R 
        Pluto sextile Pluto (orb: 5.51°) R R

        Testing date format: May 25, 2024
        Transit Report for 2024-05-25 00:00
        ==================================================

        Transiting Planets:
        --------------------
        Sun      64°0'18" (Gemini 4°0'18") 
        Moon     257°33'32" (Sagittarius 17°33'32") 
        Mercury  42°49'33" (Taurus 12°49'33") 
        Venus    61°2'59" (Gemini 1°2'59") 
        Mars     18°23'9" (Aries 18°23'9") 
        Jupiter  59°42'12" (Taurus 29°42'12") 
        Saturn   348°22'4" (Pisces 18°22'4") 
        Uranus   53°45'31" (Taurus 23°45'31") 
        Neptune  359°31'50" (Pisces 29°31'50") 
        Pluto    301°59'35" (Aquarius 1°59'35") R

        Aspects to Natal Planets:
        --------------------
        Sun conjunction Sun (orb: 0.06°)  
        Sun conjunction Venus (orb: 3.02°)  
        Sun semisquare Mars (orb: 0.68°)  
        Sun conjunction Jupiter (orb: 4.36°)  
        Sun sextile Neptune (orb: 4.53°)  
        Sun trine Pluto (orb: 2.07°)  R
        Moon opposition Sun (orb: 5.12°)  
        Mercury semisquare Mercury (orb: 1.45°)  
        Mercury semisextile Uranus (orb: 2.61°)  
        Mercury square Neptune (orb: 3.16°)  
        Venus semisextile Sun (orb: 1.33°)  
        Venus square Neptune (orb: 5.81°)  
        Mars semisextile Sun (orb: 2.83°)  
        Mars sesquisquare Moon (orb: 1.38°)  
        Mars semisextile Venus (orb: 0.13°)  
        Mars semisextile Jupiter (orb: 1.48°)  
        Mars semisextile Neptune (orb: 1.65°)  
        Mars square Pluto (orb: 0.81°)  R
        Jupiter quincunx Sun (orb: 2.79°) R 
        Jupiter opposition Mercury (orb: 6.03°) R 
        Jupiter square Pluto (orb: 4.81°) R R
        Saturn square Sun (orb: 7.73°)  
        Saturn square Moon (orb: 5.83°)  
        Saturn sextile Mercury (orb: 1.09°)  
        Saturn conjunction Saturn (orb: 6.64°)  
        Uranus trine Sun (orb: 7.91°) R 
        Uranus trine Venus (orb: 4.95°) R 
        Uranus square Mars (orb: 7.71°) R 
        Uranus trine Jupiter (orb: 3.60°) R 
        Uranus trine Uranus (orb: 2.34°) R 
        Uranus sextile Neptune (orb: 3.43°) R 
        Uranus conjunction Pluto (orb: 5.89°) R R
        Neptune trine Venus (orb: 7.94°) R 
        Neptune square Mars (orb: 4.73°) R 
        Neptune trine Jupiter (orb: 6.59°) R 
        Neptune sextile Saturn (orb: 4.75°) R 
        Neptune trine Uranus (orb: 0.64°) R 
        Pluto opposition Sun (orb: 7.52°) R 
        Pluto opposition Venus (orb: 4.57°) R 
        Pluto opposition Jupiter (orb: 3.22°) R 
        Pluto opposition Uranus (orb: 2.72°) R 
        Pluto trine Neptune (orb: 3.05°) R 
        Pluto sextile Pluto (orb: 5.51°) R R

        Testing date format: 25/05/2024
        Transit Report for 2024-05-25 00:00
        ==================================================

        Transiting Planets:
        --------------------
        Sun      64°0'18" (Gemini 4°0'18") 
        Moon     257°33'32" (Sagittarius 17°33'32") 
        Mercury  42°49'33" (Taurus 12°49'33") 
        Venus    61°2'59" (Gemini 1°2'59") 
        Mars     18°23'9" (Aries 18°23'9") 
        Jupiter  59°42'12" (Taurus 29°42'12") 
        Saturn   348°22'4" (Pisces 18°22'4") 
        Uranus   53°45'31" (Taurus 23°45'31") 
        Neptune  359°31'50" (Pisces 29°31'50") 
        Pluto    301°59'35" (Aquarius 1°59'35") R

        Aspects to Natal Planets:
        --------------------
        Sun conjunction Sun (orb: 0.06°)  
        Sun conjunction Venus (orb: 3.02°)  
        Sun semisquare Mars (orb: 0.68°)  
        Sun conjunction Jupiter (orb: 4.36°)  
        Sun sextile Neptune (orb: 4.53°)  
        Sun trine Pluto (orb: 2.07°)  R
        Moon opposition Sun (orb: 5.12°)  
        Mercury semisquare Mercury (orb: 1.45°)  
        Mercury semisextile Uranus (orb: 2.61°)  
        Mercury square Neptune (orb: 3.16°)  
        Venus semisextile Sun (orb: 1.33°)  
        Venus square Neptune (orb: 5.81°)  
        Mars semisextile Sun (orb: 2.83°)  
        Mars sesquisquare Moon (orb: 1.38°)  
        Mars semisextile Venus (orb: 0.13°)  
        Mars semisextile Jupiter (orb: 1.48°)  
        Mars semisextile Neptune (orb: 1.65°)  
        Mars square Pluto (orb: 0.81°)  R
        Jupiter quincunx Sun (orb: 2.79°) R 
        Jupiter opposition Mercury (orb: 6.03°) R 
        Jupiter square Pluto (orb: 4.81°) R R
        Saturn square Sun (orb: 7.73°)  
        Saturn square Moon (orb: 5.83°)  
        Saturn sextile Mercury (orb: 1.09°)  
        Saturn conjunction Saturn (orb: 6.64°)  
        Uranus trine Sun (orb: 7.91°) R 
        Uranus trine Venus (orb: 4.95°) R 
        Uranus square Mars (orb: 7.71°) R 
        Uranus trine Jupiter (orb: 3.60°) R 
        Uranus trine Uranus (orb: 2.34°) R 
        Uranus sextile Neptune (orb: 3.43°) R 
        Uranus conjunction Pluto (orb: 5.89°) R R
        Neptune trine Venus (orb: 7.94°) R 
        Neptune square Mars (orb: 4.73°) R 
        Neptune trine Jupiter (orb: 6.59°) R 
        Neptune sextile Saturn (orb: 4.75°) R 
        Neptune trine Uranus (orb: 0.64°) R 
        Pluto opposition Sun (orb: 7.52°) R 
        Pluto opposition Venus (orb: 4.57°) R 
        Pluto opposition Jupiter (orb: 3.22°) R 
        Pluto opposition Uranus (orb: 2.72°) R 
        Pluto trine Neptune (orb: 3.05°) R 
        Pluto sextile Pluto (orb: 5.51°) R R""",
        team_members=TEAM_MEMBERS
)
transit_node = functools.partial(agent_node, agent=transit_analyst, name="TransitAnalyst")

# Agent 4: Astrological Interpretation
astrological_interpreter = create_agent(
    llm,
    [retrieve_information],
    """You are an astrological interpreter. Your job is to interpret astrological data and provide insightful, practical advice based on the user's birth chart and transit data.
    Use the retrieve_information tool to find relevant astrological interpretations for planetary positions, aspects, and transits. Carefully apply these interpretations to the specific details of the user's birth chart and current or future transits.

    Guidelines:

    - If birth chart IS available:
        * Provide personalized interpretations using the birth chart and transits
        * Be specific: mention which planets, houses, and aspects are most relevant to the user's question.
        * Always reference the actual birth chart and transit data provided.
        * Synthesize information from the knowledge base with the user's unique astrological context.
        * Provide advice that is actionable, empathetic, and aligned with astrological best practices.
        * If the knowledge base does not contain enough information, say so and suggest what additional data would be helpful.
        * If the user’s question is about timing, highlight the most significant astrological periods and their potential effects.
        * Structure your response clearly, using headings or bullet points if helpful.

    - If NO birth chart is available:
        * Provide general astrological insights based on current transits
        * Use general astrological principles and current planetary positions
        * Mention that personalized insights would require birth details 


    Example Output:
    Astrological Analysis:
    - Jupiter trine Moon in your 6th house this month suggests increased emotional openness and support for new routines, such as adopting a pet.
    - Saturn square your 4th house cusp indicates extra responsibilities at home; plan for these before making changes.

    Advice:
    - This is a favorable time to consider new commitments, but ensure you have routines in place.
    - Reflect on how your daily habits will adapt to new responsibilities.
    If you cannot provide a complete interpretation, explain what is missing or unclear.""",
    team_members=TEAM_MEMBERS
)

astrological_node = functools.partial(agent_node, agent=astrological_interpreter, name="AstrologicalInterpreter")


### Create Supervior

In [33]:
fortuneteller_agent = create_team_supervisor(
    llm,
    ("""You are the supervisor of a fortune telling team. Your job is to coordinate between team members based on the conversation context and available information.

    Guidelines:
    - When birth details are NEEDED but NOT provided:
        * First route to AstrologicalInterpreter to:
            - Acknowledge the question
            - Explain why birth details would be helpful
            - Ask for: birth date, time, and location
            - Offer to provide general insights while waiting
        * Then route to WorldOracle for general astrological trends
        * If user provides birth details in response:
            - Route to BirthChartCalculator
        * If user doesn't provide birth details:
            - Continue with general insights

        - If birth details ARE provided:
        * Route to BirthChartCalculator first
        * Then TransitAnalyst
        * Route to WorldOracle if needed for:
            - Current events context
            - Specific life situation insights
            - Real-world trends
        * Finally AstrologicalInterpreter to synthesize everything

    Example Flow 1 (No Birth Details):
    User: "What's my love life like this year?"
    → AstrologicalInterpreter (explains need for birth details + offers general insights)
    → WorldOracle (checks current love-related transits)
    → AstrologicalInterpreter (provides general guidance)

    Example Flow 2 (Birth Details Provided Later):
    User: "What's my love life like this year?"
    → AstrologicalInterpreter (asks for details)
    User: "Born June 15, 1990, 3:30 PM in London"
    → BirthChartCalculator
    → TransitAnalyst
    → WorldOracle (if needed for context)
    → AstrologicalInterpreter

    Remember: 
    - Always maintain a conversational, helpful tone
    - Don't get stuck in loops asking for birth details
    - Provide value whether birth details are available or not
    - Route to FINISH if the question has been fully addressed"""
    ),
    ["WorldOracle", "BirthChartCalculator", "TransitAnalyst", "AstrologicalInterpreter"],
)

### Fortune-telling team graph creation

In [34]:
research_graph = StateGraph(FortuneTellingTeam)

research_graph.add_node("WorldOracle", worldoracle_node)
research_graph.add_node("BirthChartCalculator", birthchart_node)
research_graph.add_node("TransitAnalyst", transit_node)
research_graph.add_node("AstrologicalInterpreter", astrological_node)
research_graph.add_node("supervisor", fortuneteller_agent)
research_graph.set_entry_point("supervisor")

In [35]:
research_graph.add_edge("WorldOracle", "supervisor")
research_graph.add_edge("BirthChartCalculator", "supervisor")
research_graph.add_edge("TransitAnalyst", "supervisor")
research_graph.add_edge("AstrologicalInterpreter", "supervisor")

research_graph.add_conditional_edges(
    "supervisor", 
    lambda x: x["next"],
    {
        "WorldOracle": "WorldOracle",
        "BirthChartCalculator": "BirthChartCalculator",
        "TransitAnalyst": "TransitAnalyst",
        "AstrologicalInterpreter": "AstrologicalInterpreter",
        "FINISH": END,
    },
)

In [36]:
research_graph.set_entry_point("supervisor")
compiled_research_graph = research_graph.compile()

In [37]:
result = compiled_research_graph.invoke({"messages": [HumanMessage(content="What is 5th house?")], "next": "supervisor"})

In [38]:
result

{'messages': [HumanMessage(content='What is 5th house?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='The 5th house in astrology is the house of creativity, pleasure, and self-expression. It governs areas such as:\n\n- Children and your relationship with them\n- Creative pursuits like art, drama, hobbies, and teaching\n- Love affairs, romance, dating, and casual relationships (distinct from marriage, which is ruled by the 7th house)\n- Personal enjoyment, fun, sports, gambling, and risk-taking\n- Speculation and playful self-expression\n\nEssentially, the 5th house represents how you enjoy life, express your individuality, and seek joy and passion. It highlights the playful, romantic, and creative side of your personality.\n\nIf you want a personalized interpretation of the 5th house in your chart or how current or upcoming transits might affect this area, feel free to provide your birth details or specific planetary placements.', additional_kwargs={}, response_

In [39]:
def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
    }
    return results

rag_chain = enter_chain | compiled_research_graph

In [40]:
for s in rag_chain.stream(
    "What is Mars in 20th house?", {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'AstrologicalInterpreter'}}
---
{'AstrologicalInterpreter': {'messages': [HumanMessage(content='Astrological Interpretation: Mars in the 20th House\n\nThe traditional astrological system recognizes only 12 houses in a birth chart, so there is no standard or classical "20th house." The concept of a 20th house might come from a specialized or non-traditional house system, a harmonic chart division, or could be a misunderstanding or typo.\n\nMars represents initiative, physical energy, drive, assertiveness, and how you take action. It can bring ambition and assertiveness but also potential for anger or impulsiveness.\n\nSince the 20th house is not recognized in typical astrology, I can\'t provide a direct interpretation for Mars there. If you meant Mars in the 2nd or 10th house (which are standard houses), I can offer detailed insights on those.\n\nIf you have more information about the house system or chart type you are using, please share it, and I can help inter

In [41]:
for s in rag_chain.stream(
    "I was born on 13 May 1990 at 12:00 in Paris, I wanna know about my life", {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'BirthChartCalculator'}}
---
{'BirthChartCalculator': {'messages': [HumanMessage(content='Location: Paris\nLatitude: 48.8588897°N\nLongitude: 2.320041°E\nTime: 1990-05-13 12:00 (UTC+2.0)\nJulian Day: 2448024.9166666665\n\nPlanetary Positions:\n------------------\nSun      52°23\'11" (Taurus 22°23\'11")\nMoon     271°49\'33" (Capricorn 1°49\'33")\nMercury  38°25\'23" (Taurus 8°25\'23")\nVenus    10°26\'44" (Aries 10°26\'44")\nMars     346°47\'8" (Pisces 16°47\'8")\nJupiter  99°9\'0" (Cancer 9°9\'0")\nSaturn   295°16\'52" (Capricorn 25°16\'52")\nUranus   279°14\'1" (Capricorn 9°14\'1")\nNeptune  284°22\'57" (Capricorn 14°22\'57")\nPluto    226°13\'31" (Scorpio 16°13\'31")\n\nAscendant (Rising Sign):\nPosition: 131°38\'33" (Leo 11°38\'33")\n\nHouse Cusps:\n------------\nhouse_1  131°38\'33" (Leo 11°38\'33")\nhouse_2  150°4\'17" (Virgo 0°4\'17")\nhouse_3  173°39\'52" (Virgo 23°39\'52")\nhouse_4  205°3\'50" (Libra 25°3\'50")\nhouse_5  243°52\'40" (Sagittarius 3°52\'4

## Creating a Golden Test Dataset

In [42]:
#RAGAS
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/keatnuxsuo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/keatnuxsuo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [43]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = "cyberteller"

### Using the same html files to generate synthetic data

In [44]:
docs

[Document(metadata={'source': 'data/Synastry Art and Science- Synastry Aspects, Element Compatibility, House Overlays.html'}, page_content='Astrolibrary logo\n\nAstroLibrary 🔎\n\nDaily Personalized Horoscope ↗\n\nSynastry Art and Science- Synastry Aspects, Element Compatibility, House Overlays\n\n(This is a synastry lesson. If you prefer, see Synastry Interpretations instead.)\n\nWhat is synastry? Synastry is the art and science of comparing two birth charts to ascertain the energy flow between two individuals. Synastry is the astrology of relationships, and it reveals the level of compatibility between two people. Please don\'t confuse this with fortune-telling. Synastry will not tell you if you are "fated to be together." Any two people can choose to be together. Synastry shows if two unique individuals can be themselves and blend nicely together, or if being themselves causes the irritating grinding of mismatched gears.\n\nSpecifically, 3 things are compared:\n\nElements – At the ve

### Knowledge Graph Based Synthetic Generation

In [45]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [46]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

In [47]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 15, relationships: 0)

In [48]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/15 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/15 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/20 [00:00<?, ?it/s]

Property 'summary' already exists in node 'df070f'. Skipping!
Property 'summary' already exists in node '2c2289'. Skipping!
Property 'summary' already exists in node '044222'. Skipping!
Property 'summary' already exists in node '75c060'. Skipping!
Property 'summary' already exists in node '84b8a4'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/37 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/76 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'df070f'. Skipping!
Property 'summary_embedding' already exists in node '044222'. Skipping!
Property 'summary_embedding' already exists in node '75c060'. Skipping!
Property 'summary_embedding' already exists in node '84b8a4'. Skipping!
Property 'summary_embedding' already exists in node '2c2289'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 48, relationships: 397)

In [49]:
kg.save("astrology_101.json")
astrology_101 = KnowledgeGraph.load("astrology_101.json")
astrology_101

KnowledgeGraph(nodes: 48, relationships: 397)

### Generate 3 different varities of test data

In [50]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=astrology_101)

In [51]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

In [52]:
dataset = generator.generate(testset_size=10, query_distribution=query_distribution)
dataset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the significance of the 1st House cusp...,[What is a Rising Sign/Ascendant – Astrology L...,The 1st House cusp in a natal chart is where t...,single_hop_specifc_query_synthesizer
1,What key advice does Lesson 8: Zodiac Signs of...,[Astrolibrary logo AstroLibrary 🔎 Daily Person...,Lesson 8: Zodiac Signs emphasizes that the mos...,single_hop_specifc_query_synthesizer
2,what 2nd House mean in chart?,[Incorporating the Astrological Houses The oth...,The 2nd House is an area of life where the rul...,single_hop_specifc_query_synthesizer
3,"How does the term ""Henley"" relate to the inter...",[A Word of Caution The celestial configuration...,"The term ""Henley"" refers to the quote, ""I am t...",single_hop_specifc_query_synthesizer
4,How does Neptune influence inner planets when ...,[Remembering Aspects Don't forget the aspects....,When Neptune makes a conjunction aspect with a...,single_hop_specifc_query_synthesizer
5,How does the unique combination of elements an...,[<1-hop>\n\nWater Zodiac Element Water signs r...,The unique combination of elements and qualiti...,multi_hop_abstract_query_synthesizer
6,"How do the major aspects such as conjunction, ...",[<1-hop>\n\nAstrolibrary logo AstroLibrary 🔎 D...,The major aspects in astrology—conjunction (0°...,multi_hop_abstract_query_synthesizer
7,how interaction of planetary placements and ex...,[<1-hop>\n\nAstrolibrary logo AstroLibrary 🔎 D...,The interaction of planetary placements in a b...,multi_hop_abstract_query_synthesizer
8,How does the sign of Capricorn influence the e...,[<1-hop>\n\nsign of Mars in your natal chart i...,The sign of Capricorn influences the expressio...,multi_hop_specific_query_synthesizer
9,How does havin Saturn in Capricorn in the 7th ...,[<1-hop>\n\nIncorporating the Astrological Hou...,Having Saturn in Capricorn in the 7th House ca...,multi_hop_specific_query_synthesizer


In [53]:
#response = graph.invoke({"question" : "What does Mars in 5th mean?"})

In [54]:
response

{'question': 'What does Mars in 5th mean?',
 'context': [Document(metadata={'source': 'data/Planets in Astrology – Lesson 5 _ AstroLibrary.html', '_id': '0af75eef9c8c4a9b8684f989fbf470e1', '_collection_name': 'astrology101'}, page_content="Interpretations for Venus in each zodiac sign are available here: Venus in the Signs\n\nInterpretations for Venus in each House of the birth chart: Venus in Houses\n\nReturn to Top Return to Taurus Return to Libra\n\nastrology planet mars real image\n\nMars\n\n“I act”\n\nBest quality: Initiative Worst quality: Harshness\n\nMars is the exact opposite of Venus. Whereas Venus represents the female, Mars is all male. Whereas Venus is about harmony, Mars is conflict, aggression, and outright war. Mars is the planet of physical energy. It governs your sex drive, your forcefulness and your aggression. It is associated with your desires and aspirations for emotional, physical, mental and spiritual things that stimulate activity. You have to want something be

### Using RAG Chain to answer test questions and add generated answers into the dataframe

In [55]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [56]:
# check the updated dataset
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,What is the significance of the 1st House cusp...,[House Rulers (Signs on the Cusps)\n\nAnother ...,[What is a Rising Sign/Ascendant – Astrology L...,Certainly! Here is a detailed astrological int...,The 1st House cusp in a natal chart is where t...,single_hop_specifc_query_synthesizer
1,What key advice does Lesson 8: Zodiac Signs of...,[Astrolibrary logo\n\nAstroLibrary 🔎\n\nDaily ...,[Astrolibrary logo AstroLibrary 🔎 Daily Person...,Certainly! Based on Lesson 8: Zodiac Signs fro...,Lesson 8: Zodiac Signs emphasizes that the mos...,single_hop_specifc_query_synthesizer
2,what 2nd House mean in chart?,"[Simply put, the 8th house is the polar opposi...",[Incorporating the Astrological Houses The oth...,Certainly! Here is a detailed astrological int...,The 2nd House is an area of life where the rul...,single_hop_specifc_query_synthesizer
3,"How does the term ""Henley"" relate to the inter...",[Some astrologers mistakenly interpret birth c...,[A Word of Caution The celestial configuration...,"The term **""Henley""** in the provided context ...","The term ""Henley"" refers to the quote, ""I am t...",single_hop_specifc_query_synthesizer
4,How does Neptune influence inner planets when ...,[Remembering Aspects\n\nDon't forget the aspec...,[Remembering Aspects Don't forget the aspects....,When **Neptune forms a conjunction aspect with...,When Neptune makes a conjunction aspect with a...,single_hop_specifc_query_synthesizer
5,How does the unique combination of elements an...,[Numerous astrological techniques can (and sho...,[<1-hop>\n\nWater Zodiac Element Water signs r...,Certainly! Let’s explore how the unique combin...,The unique combination of elements and qualiti...,multi_hop_abstract_query_synthesizer
6,"How do the major aspects such as conjunction, ...",[Example of an Aspect\n\nastrology chart squar...,[<1-hop>\n\nAstrolibrary logo AstroLibrary 🔎 D...,Certainly! Here’s a detailed astrological inte...,The major aspects in astrology—conjunction (0°...,multi_hop_abstract_query_synthesizer
7,how interaction of planetary placements and ex...,[Remembering Aspects\n\nDon't forget the aspec...,[<1-hop>\n\nAstrolibrary logo AstroLibrary 🔎 D...,Certainly! Here’s a detailed explanation of ho...,The interaction of planetary placements in a b...,multi_hop_abstract_query_synthesizer
8,How does the sign of Capricorn influence the e...,[The zodiac sign will show how the planet is m...,[<1-hop>\n\nsign of Mars in your natal chart i...,Certainly! Here’s a detailed astrological inte...,The sign of Capricorn influences the expressio...,multi_hop_specific_query_synthesizer
9,How does havin Saturn in Capricorn in the 7th ...,[House Rulers (Signs on the Cusps)\n\nAnother ...,[<1-hop>\n\nIncorporating the Astrological Hou...,Certainly! Here is a detailed astrological int...,Having Saturn in Capricorn in the 7th House ca...,multi_hop_specific_query_synthesizer


### Evaluation using RAGAS

In [57]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

In [58]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

In [59]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception raised in Job[5]: TimeoutError()
Exception raised in Job[35]: TimeoutError()
Exception raised in Job[41]: TimeoutError()
Exception raised in Job[47]: TimeoutError()


In [60]:
result

{'context_recall': 0.9583, 'faithfulness': 0.7147, 'factual_correctness(mode=f1)': 0.6091, 'answer_relevancy': 0.9484, 'context_entity_recall': 0.5928, 'noise_sensitivity(mode=relevant)': 0.2707}

## Use Finetuning Model

In [61]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="DiamondCutter88/astrology-ft-c8cd64c7-86a1-4203-ac38-826b510e436f")

Some weights of BertModel were not initialized from the model checkpoint at DiamondCutter88/astrology-ft-c8cd64c7-86a1-4203-ac38-826b510e436f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [62]:
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore_fine_tuned = Qdrant.from_documents(
    split_chunks,
    huggingface_embeddings,
    location=":memory:",
    collection_name="fine-tuned-astrology101"
)

In [63]:
qdrant_retriever_fine_tuned  = qdrant_vectorstore_fine_tuned.as_retriever(search_kwargs={"k": 5})

In [64]:
def retrieve_fine_tuned(state):
  retrieved_docs = qdrant_retriever_fine_tuned.invoke(state["question"])
  return {"context" : retrieved_docs}

Build Finetuned retriever graph

In [65]:
graph_builder = StateGraph(State)
# Define nodes
graph_builder.add_node("retrieve_fine_tuned", retrieve_fine_tuned)
graph_builder.add_node("generate", generate)
# Define edges
graph_builder.add_edge(START, "retrieve_fine_tuned")
graph_builder.add_edge("retrieve_fine_tuned", "generate")
graph_builder.add_edge("generate", END)
# Compile
graph_retrieve_fine_tuned = graph_builder.compile()

In [66]:
response_retrieve_fine_tuned = graph_retrieve_fine_tuned.invoke({"question" : "What does Mars in 5th mean?"})
response_retrieve_fine_tuned

{'question': 'What does Mars in 5th mean?',
 'context': [Document(metadata={'source': 'data/Planets in Astrology – Lesson 5 _ AstroLibrary.html', '_id': '59eb5ed06f17481e8156a6f8725e2b90', '_collection_name': 'fine-tuned-astrology101'}, page_content="Interpretations for Venus in each zodiac sign are available here: Venus in the Signs\n\nInterpretations for Venus in each House of the birth chart: Venus in Houses\n\nReturn to Top Return to Taurus Return to Libra\n\nastrology planet mars real image\n\nMars\n\n“I act”\n\nBest quality: Initiative Worst quality: Harshness\n\nMars is the exact opposite of Venus. Whereas Venus represents the female, Mars is all male. Whereas Venus is about harmony, Mars is conflict, aggression, and outright war. Mars is the planet of physical energy. It governs your sex drive, your forcefulness and your aggression. It is associated with your desires and aspirations for emotional, physical, mental and spiritual things that stimulate activity. You have to want s

In [67]:
for test_row in dataset:
  response = graph_retrieve_fine_tuned.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [68]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,What is the significance of the 1st House cusp...,[Astrolibrary logo\n\nAstroLibrary 🔎\n\nDaily ...,[What is a Rising Sign/Ascendant – Astrology L...,Certainly! Here is a detailed astrological int...,The 1st House cusp in a natal chart is where t...,single_hop_specifc_query_synthesizer
1,What key advice does Lesson 8: Zodiac Signs of...,[Astrolibrary logo\n\nAstroLibrary 🔎\n\nDaily ...,[Astrolibrary logo AstroLibrary 🔎 Daily Person...,I don’t have enough information as my cosmic s...,Lesson 8: Zodiac Signs emphasizes that the mos...,single_hop_specifc_query_synthesizer
2,what 2nd House mean in chart?,"[Simply put, the 8th house is the polar opposi...",[Incorporating the Astrological Houses The oth...,Certainly! Here’s a detailed astrological inte...,The 2nd House is an area of life where the rul...,single_hop_specifc_query_synthesizer
3,"How does the term ""Henley"" relate to the inter...",[Some astrologers mistakenly interpret birth c...,[A Word of Caution The celestial configuration...,Certainly! Based on the astrological context p...,"The term ""Henley"" refers to the quote, ""I am t...",single_hop_specifc_query_synthesizer
4,How does Neptune influence inner planets when ...,[Remembering Aspects\n\nDon't forget the aspec...,[Remembering Aspects Don't forget the aspects....,**How Does Neptune Influence Inner Planets Whe...,When Neptune makes a conjunction aspect with a...,single_hop_specifc_query_synthesizer
5,How does the unique combination of elements an...,[Astrolibrary logo\n\nAstroLibrary 🔎\n\nDaily ...,[<1-hop>\n\nWater Zodiac Element Water signs r...,Certainly! Here’s a detailed astrological inte...,The unique combination of elements and qualiti...,multi_hop_abstract_query_synthesizer
6,"How do the major aspects such as conjunction, ...",[Remembering Aspects\n\nDon't forget the aspec...,[<1-hop>\n\nAstrolibrary logo AstroLibrary 🔎 D...,Certainly! Here is a detailed explanation of h...,The major aspects in astrology—conjunction (0°...,multi_hop_abstract_query_synthesizer
7,how interaction of planetary placements and ex...,[Remembering Aspects\n\nDon't forget the aspec...,[<1-hop>\n\nAstrolibrary logo AstroLibrary 🔎 D...,Certainly! Here’s a detailed explanation of ho...,The interaction of planetary placements in a b...,multi_hop_abstract_query_synthesizer
8,How does the sign of Capricorn influence the e...,[Interpretations for Venus in each zodiac sign...,[<1-hop>\n\nsign of Mars in your natal chart i...,Certainly! Here is a detailed astrological int...,The sign of Capricorn influences the expressio...,multi_hop_specific_query_synthesizer
9,How does havin Saturn in Capricorn in the 7th ...,"[Nowadays, it's common to marry for love. To a...",[<1-hop>\n\nIncorporating the Astrological Hou...,Certainly! Let’s explore the astrological sign...,Having Saturn in Capricorn in the 7th House ca...,multi_hop_specific_query_synthesizer


In [69]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

finetuned_evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
custom_run_config = RunConfig(timeout=360)

finetuned_result = evaluate(
    dataset=finetuned_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception raised in Job[5]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[29]: TimeoutError()
Exception raised in Job[35]: TimeoutError()
Exception raised in Job[41]: TimeoutError()
Exception raised in Job[47]: TimeoutError()


In [70]:
finetuned_result 

{'context_recall': 0.7705, 'faithfulness': 0.7452, 'factual_correctness(mode=f1)': 0.5564, 'answer_relevancy': 0.8669, 'context_entity_recall': 0.5315, 'noise_sensitivity(mode=relevant)': 0.2461}

In [77]:
import pandas as pd

# Convert both results to pandas DataFrames
finetuned_df = finetuned_result.to_pandas()
baseline_df = result.to_pandas()

# Calculate mean scores for each metric
metrics = [
    'context_recall', 'faithfulness', 'factual_correctness(mode=f1)',
    'answer_relevancy', 'context_entity_recall', 'noise_sensitivity(mode=relevant)'
]

comparison_df = pd.DataFrame({
    'Metric': metrics,
    'Finetuned Model': [finetuned_df[metric].mean() for metric in metrics],
    'Baseline Model': [baseline_df[metric].mean() for metric in metrics]
})

# Format the values as percentages
comparison_df['Finetuned Model'] = comparison_df['Finetuned Model'].apply(lambda x: f'{x:.2%}')
comparison_df['Baseline Model'] = comparison_df['Baseline Model'].apply(lambda x: f'{x:.2%}')

# Display the DataFrame
print("\nModel Comparison Metrics (Average Scores):")
display(comparison_df)


Model Comparison Metrics (Average Scores):


,Metric,Finetuned Model,Baseline Model
0,context_recall,77.05%,95.83%
1,faithfulness,74.52%,71.47%
2,factual_correctness(mode=f1),55.64%,60.91%
3,answer_relevancy,86.69%,94.84%
4,context_entity_recall,53.15%,59.28%
5,noise_sensitivity(mode=relevant),24.61%,27.07%


1. **Context Recall** (Baseline: 95.83% vs Finetuned: 77.05%)
   - The baseline model is significantly better at retrieving relevant context
   - This suggests the finetuning process may have made the model more selective or less comprehensive in its context retrieval
   - Example: In the "Henley" question, the baseline model better captured the full context about free will vs fate

2. **Faithfulness** (Finetuned: 74.52% vs Baseline: 71.47%)
   - The finetuned model performs slightly better here
   - This suggests it's better at staying true to the source content
   - However, the improvement is marginal

3. **Factual Correctness** (Baseline: 60.91% vs Finetuned: 55.64%)
   - The baseline model is more accurate in its factual responses
   - The finetuned model shows more errors in factual information
   - Example: In the "2nd House" question, the baseline model provided more accurate astrological interpretations

4. **Answer Relevancy** (Baseline: 94.84% vs Finetuned: 86.69%)
   - The baseline model's answers are more relevant to the questions
   - The finetuned model sometimes goes off-topic or provides less focused responses
   - Example: In the "Neptune" question, the baseline model stayed more focused on the specific aspect being asked about

5. **Context Entity Recall** (Baseline: 59.28% vs Finetuned: 53.15%)
   - The baseline model is better at recalling specific entities from the context
   - The finetuned model misses some key entities or concepts

6. **Noise Sensitivity** (Baseline: 27.07% vs Finetuned: 24.61%)
   - Both models show relatively low noise sensitivity
   - The baseline model is slightly more sensitive to noise

**Potential Reasons for Worse Performance:**

1. **Overfitting**: The finetuned model may have overfit to specific patterns in the training data, making it less generalizable

2. **Loss of General Knowledge**: The finetuning process might have caused the model to lose some of its general knowledge or ability to retrieve broad context

3. **Narrow Focus**: The finetuning may have made the model too specialized, causing it to miss important context or entities

4. **Training Data Quality**: The quality or size of the finetuning dataset might not have been optimal

**Recommendations for Improvement:**

1. **Review Finetuning Data**:
   - Ensure the training data is diverse and high-quality
   - Include more examples of different types of astrological questions

2. **Adjust Finetuning Parameters**:
   - Try different learning rates
   - Experiment with different numbers of training epochs
   - Consider using a smaller model as the base

3. **Balance Specialization**:
   - Ensure the finetuning process maintains a balance between specialization and general knowledge
   - Consider using techniques like LoRA or QLoRA to preserve more of the base model's capabilities

4. **Evaluation Metrics**:
   - Add more specific evaluation metrics for astrological accuracy
   - Consider using human evaluation for complex astrological interpretations
